In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import os
import os.path as op
import mne
# from mne_bids import make_bids_basename
from IPython.display import Image
from mayavi import mlab

from nilearn import plotting

mne.set_log_level('WARNING')

In [2]:
data_dir = op.expanduser("~/data/pic-name-data-bids/")
sSub, session, task, run = '15', '01', 'picturenaming', '01'

data_path = op.join(data_dir, 'MEG')
subjects_dir = op.join(data_dir, 'MRI')
subject = 'sub-' + sSub

bids_basename = subject + '_ses-' + session + '_task-' + task + '_run-' + run
bids_path = op.join(data_path, subject, 'ses-'+session, 'meg')

raw_fname = op.join(bids_path, bids_basename + '_meg.fif')

t1_fname = os.path.join(subjects_dir, subject, 'mri/T1.mgz')

In [ ]:
plotting.plot_anat(t1_fname);
plt.show()

In [ ]:
mne.viz.plot_bem(subject=subject, subjects_dir=subjects_dir,
                orientation='coronal')

In [ ]:
mne.gui.coregistration(subject=subject, subjects_dir=subjects_dir, inst=raw_fname);

In [3]:
trans_fname = os.path.join(bids_path, subject + '-trans.fif')
info = mne.io.read_info(raw_fname)

In [ ]:
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                            subjects_dir=subjects_dir, verbose=True)

In [ ]:
mlab.close('all')

In [4]:
src = mne.setup_source_space(subject, spacing='oct6', subjects_dir=subjects_dir,
                            add_dist=False)

In [5]:
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, src=src,
                            subjects_dir=subjects_dir, verbose=True, meg=False,
                            eeg=False);

Using outer_skin.surf for head surface.
Using pyvista 3d backend.



In [6]:
src

<SourceSpaces: [<surface (lh), n_vertices=97538, n_used=4098>, <surface (rh), n_vertices=97577, n_used=4098>] MRI (surface RAS) coords, subject 'sub-15', ~15.1 MB>

In [ ]:
mne.viz.plot_alignment(info, trans_fname, subject=subject,
                      src=src, subjects_dir=subjects_dir, dig=True,
                      surfaces=['head', 'white'], coord_frame='meg', meg=False)

In [ ]:
# mlab.close()

In [7]:
conductivity = (0.3,)
model = mne.make_bem_model(subject=subject, ico=4, conductivity=conductivity,
                          subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [8]:
model

[{'id': 1 (FIFFV_BEM_SURF_ID_BRAIN),
  'sigma': 0.3,
  'np': 2562,
  'ntri': 5120,
  'coord_frame': array([5], dtype=int32),
  'rr': array([[-3.85086089e-05, -2.39945679e-02,  8.05411758e-02],
         [ 6.86019135e-02, -2.30213127e-02,  4.26053085e-02],
         [ 2.25956802e-02,  5.21331749e-02,  4.69905968e-02],
         ...,
         [ 5.36347198e-03, -1.59341488e-02, -4.60927086e-02],
         [ 9.97709632e-04, -1.45753307e-02, -4.52835312e-02],
         [ 1.80792975e-03, -1.87436543e-02, -4.93118629e-02]]),
  'nn': array([[-0.01363403,  0.03370834,  0.99933871],
         [ 0.91342325,  0.0353694 ,  0.40547129],
         [ 0.41399455,  0.64743166,  0.63987558],
         ...,
         [ 0.21508417,  0.68290601, -0.69812476],
         [-0.02725725,  0.70398103, -0.70969554],
         [ 0.01549116,  0.6258422 , -0.77979585]]),
  'tris': array([[   0,  644,  642],
         [ 642,  643,  162],
         [ 644,  643,  642],
         ...,
         [2561, 2201,  557],
         [2477, 2201,

In [ ]:
bem_fname = os.path.join(bids_path, subject + '-bem.fif')
mne.bem.write_bem_solution(bem_fname, bem)

In [15]:
fwd = mne.make_forward_solution(raw_fname, trans=trans_fname, src=src,
                               bem=bem, meg=True, eeg=False, mindist=5.0, n_jobs=1)

In [12]:
# fwd_fname = os.path.join(bids_path, subject + '-meg-fwd.fif')
# mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

In [14]:
# fwd = mne.read_forward_solution(fwd_fname)
# fwd

In [16]:
fwd = mne.convert_forward_solution(fwd, surf_ori=True)
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 306 sensors x 24588 dipoles


In [ ]:
sens_map = mne.sensitivity_map(fwd, ch_type='grad', mode='fixed')

In [ ]:
# picks = mne.pick_types(fwd['info'], meg='grad', eeg=False)
# im = plt.imshow(leadfield[picks, :500], origin='lower', aspect='auto', cmap='RdBu_r')
# plt.xlabel('sources')
# plt.ylabel('sensors')
# plt.title('Leadfield matrix for Gradiometers', fontsize=14)
# plt.colorbar(cmap='RdBu_r')

In [ ]:
# plt.figure()
# plt.hist(sens_map.data.ravel(), bins=20, label='Gradiometers', color='c')
# plt.legend()
# plt.title('Normal orientation sensitivity')
# plt.xlabel('sensitivity')
# plt.ylabel('count');

In [ ]:
# clim = dict(kind='percent', lims=(0, 50, 99), smoothing_steps=3)
# brain = sens_map.plot(subject=subject, time_label='GRAD sensitivity',
#                      subjects_dir=subjects_dir, clim=clim, smoothing_steps=8);
# view = 'lat'
# brain.show_view(view)

In [ ]:
# brain.save_image('sensitivity_map_grad_%s.jpg' % view)
# Image(filename='sensitivity_map_grad_%s.jpg' % view, width=400)